In [1]:
from parflowio.pyParflowio import PFData
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sx_pfb = PFData('/home/ltelfer/scratch/upper_sfs/input_files/domain_inputs/usfs_slopex.pfb')
sx_pfb.loadHeader();
sx_pfb.loadData();
sx_arr = sx_pfb.copyDataArray().squeeze()
sx = xr.DataArray(sx_arr,dims=['y','x'],name='slopex')

sy_pfb = PFData('/home/ltelfer/scratch/upper_sfs/input_files/domain_inputs/usfs_slopey.pfb')
sy_pfb.loadHeader();
sy_pfb.loadData();
sy_arr = sy_pfb.copyDataArray().squeeze()
sy = xr.DataArray(sy_arr,dims=['y','x'],name='slopex')

ctr = xr.open_mfdataset('/home/ltelfer/scratch/upper_sfs/pfout.zarr/ctr/ctr',engine='zarr')
mask = ctr.mask.sel(z=25).drop_vars('z')

In [3]:
slope = (sx+sy).where(mask==1)
mslope = abs(slope).quantile(0.5).values.tolist()
slope_uniform = (slope/abs(slope))*mslope
sx_abs = abs(sx.where(mask==1))
sy_abs = abs(sy.where(mask==1))
sx_uniform = slope_uniform.where((mask==1)&(sx_abs>0),0)
sy_uniform = slope_uniform.where((mask==1)&(sy_abs>0),0)
sx_uniform = np.expand_dims(sx_uniform.where(mask==1,-999.).values,0)
sy_uniform = np.expand_dims(sy_uniform.where(mask==1,-999.).values,0)

In [4]:
sx_pfb.setDataArray(sx_uniform)
sx_pfb.writeFile('/home/ltelfer/scratch/upper_sfs/input_files/domain_inputs/uniform_slopex.pfb')

0

In [5]:
sy_pfb.setDataArray(sy_uniform)
sy_pfb.writeFile('/home/ltelfer/scratch/upper_sfs/input_files/domain_inputs/uniform_slopey.pfb')

0